# 此程序用来求解正方形上Helmholtz方程的格林函数

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable as v
import scipy.special as scp

is_gpu = torch.cuda.is_available()
if is_gpu:
    id = 0
    torch.cuda.set_device(id)
    
#gpu_nums = torch.cuda.device_count()
#gpu_index = torch.cuda.current_device()
#print(is_gpu,gpu_nums,gpu_index)
device = torch.device('cuda' if is_gpu else 'cpu')

device = torch.device('cpu')
print(device)
torch.set_default_tensor_type('torch.DoubleTensor')
print(torch.__version__)


cpu
1.6.0+cu101


In [4]:
# 设置多边形区域和边界上的节点和边界条件 设置积分矩阵
# vertex_num  多边形边数
# vertex      多边形顶点（按逆时针连接）
# M           在每条边上取的样本点数
# normal      样本点处的外法向量
# h           每条边上两点间的距离
# sample_num  总样本点数即 每条边上样本点数*多边形顶点数
# sample_x    样本点坐标
# k           波数

# G2_r        基本解的外法向导数的实部
# G2_i        基本解的外法向导数的虚部
# A2_r        积分矩阵的实部
# A2_i        积分矩阵的虚部

# in_xy       网络的输入即(x,y)两点的product
# sample_u_r  边界条件的实部
# sample_u_i  边界条件的虚部

k=1
M = 50
line = (torch.linspace(0,1-1/M,M)).reshape(-1,1)
vertex_num = 4
vertex = torch.Tensor([[-1,-1],[1,-1],[1,1],[-1,1],[-1,-1]])

rot = torch.Tensor([[0,-1],[1,0]])  #顺时针旋转90度

normal = torch.zeros(vertex_num,2)  #各边法向量
h = torch.zeros(vertex_num,1)     #各边小区间单位长度
for i in range(vertex_num):
    normal[i,:] = (vertex[i+1,:]-vertex[i,:])@rot
    normal[i,:] = normal[i,:]/(normal[i,:].norm())
    h[i] = (vertex[i+1,:]-vertex[i,:]).norm()/M

sample_num = vertex_num*M
sample_x = torch.zeros(vertex_num*M,2)
for i in range(vertex_num):
    sample_x[i*M:i*M+M,:] = vertex[i,:]+(vertex[i+1,:]-vertex[i,:])*line
sample_u = torch.zeros(sample_num,1)

G2_r = torch.zeros(sample_num,sample_num)
G2_i = torch.zeros(sample_num,sample_num)
for i in range(sample_num):
    for j in range(sample_num):
        d = sample_x[j,:] - sample_x[i,:]
        r0 = d.norm()
        j0 = int(j/M)
        j1 = int((j-1)/M)%vertex_num
        G2_r[i,j] = scp.hankel1(1,k*r0).imag/4*((d*normal[j0,:]*h[j0]+d*normal[j1,:]*h[j1]).sum())/2/r0*k 
        G2_i[i,j] = -scp.hankel1(1,k*r0).real/4*((d*normal[j0,:]*h[j0]+d*normal[j1,:]*h[j1]).sum())/2/r0*k 
        
A2_r = torch.zeros(sample_num,sample_num)
A2_i = torch.zeros(sample_num,sample_num)
for i in range(sample_num):
    if (i%M != 0) and (i%M != 1) and (i%M != M-1):
        for j in range(sample_num):
            if i==j:
                A2_r[i,j] , A2_i[i,j] = 1/2 , 0
            else:
                A2_r[i,j] , A2_i[i,j] = -G2_r[i,j] , -G2_i[i,j]

M0 = 120
in_x = torch.rand(M0,2)*2-1
int_xy = torch.zeros(M0*sample_num,4)
for i in range(M0):
    int_xy[i*sample_num:(i+1)*sample_num,0:2] = in_x[i,:]
    int_xy[i*sample_num:(i+1)*sample_num,2:4] = sample_x
sample_u_r = -scp.hankel1(0,k*(((int_xy[:,0:2]-int_xy[:,2:4])**2).sum(axis=1)).sqrt()).imag/4
sample_u_i = scp.hankel1(0,k*(((int_xy[:,0:2]-int_xy[:,2:4])**2).sum(axis=1)).sqrt()).real/4
for i in range(M0*vertex_num):
    sample_u_r[i*M] , sample_u_i[i*M] = 0 , 0
    sample_u_r[i*M+1] , sample_u_i[i*M+1] = 0 , 0
    sample_u_r[i*M+M-1] , sample_u_i[i*M+M-1] = 0 , 0

In [5]:
# 网络结构

# ResNet结构
# m 为神经元个数
m = 60
class Net(nn.Module):
  def __init__(self,m,out):
    super(Net, self).__init__()
    self.input = nn.Linear(4,m)
    self.block1=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block2=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block3=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block4=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block5=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block6=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block7=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.block8=nn.Sequential(
      nn.Linear(m,m),nn.ReLU(),
      nn.Linear(m,m),nn.ReLU(),
    )
    self.out = nn.Linear(m,out)
  def forward(self, x):
      x = self.input(x)
      x = self.block1(x) + x
      x = self.block2(x) + x
      x = self.block3(x) + x
      x = self.block4(x) + x
      x = self.block5(x) + x
      x = self.block6(x) + x
      x = self.block7(x) + x
      x = self.block8(x) + x
      x = self.out(x)
      return x

net1 = Net(m,2) #用格林函数形式作为损失函数

In [7]:
#训练过程

# optimizer   优化器
# Epoch       总训练次数
# int_h_r  样本点处密度函数值h(x)的实部
# int_h_i  样本点处密度函数值h(x)的虚部
# u0_r        样本点处格林函数数值解实部
# u0_i        样本点处格林函数数值解虚部
optimizer = torch.optim.Adam(net1.parameters(net1),lr=0.0001)
Epoch = 100
loss_all = np.zeros(Epoch+1)
u0 = torch.zeros(M0*sample_num,1)
for epoch in range(Epoch+1):
    loss = 0
    
    if (epoch)%200==0:  
        M0 = 120
        in_x = torch.rand(M0,2)*2-1
        int_xy = torch.zeros(M0*sample_num,4)
        for i in range(M0):
            int_xy[i*sample_num:(i+1)*sample_num,0:2] = in_x[i,:]
            int_xy[i*sample_num:(i+1)*sample_num,2:4] = sample_x
        sample_u_r = -scp.hankel1(0,k*(((int_xy[:,0:2]-int_xy[:,2:4])**2).sum(axis=1)).sqrt()).imag/4
        sample_u_i = scp.hankel1(0,k*(((int_xy[:,0:2]-int_xy[:,2:4])**2).sum(axis=1)).sqrt()).real/4
        for i in range(M0*vertex_num):
            sample_u_r[i*M] , sample_u_i[i*M] = 0 , 0
            sample_u_r[i*M+1] , sample_u_i[i*M+1] = 0 , 0
            sample_u_r[i*M+M-1] , sample_u_i[i*M+M-1] = 0 , 0
        int_xy = int_xy.to(device)
        sample_u_r = sample_u_r.to(device)
        sample_u_i = sample_u_i.to(device) 
    
    int_h_r,int_h_i = net1(int_xy)[:,0] , net1(int_xy)[:,1]#这个方法甚至不需要内部点
    for i in range(M0):
        u0_r = A2_r@(int_h_r[i*sample_num:(i+1)*sample_num])-A2_i@(int_h_i[i*sample_num:(i+1)*sample_num])
        u0_i = A2_i@(int_h_r[i*sample_num:(i+1)*sample_num])+A2_r@(int_h_i[i*sample_num:(i+1)*sample_num])
        loss = loss + torch.mean(torch.pow(u0_r-sample_u_r[i*sample_num:(i+1)*sample_num],2)+torch.pow(u0_i-sample_u_i[i*sample_num:(i+1)*sample_num],2))
            
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_all[epoch] = loss.cpu().detach().numpy() 
    if epoch%100==0:
        print('loss, epoch:', loss.detach().numpy(),epoch)


loss, epoch: 38.387561366955836 0
loss, epoch: 0.6338183372392864 100


## 下面的部分是用上面求解得到的格林函数求解方程

In [8]:
### 设置积分节点 ###

# 设置积分的权重（利用高斯积分的product形式）
int_sample_num = 20 
h0 = 2/int_sample_num
int_num = 2

vertex_x = torch.Tensor([[-1,-1],[1,-1],[1,1],[-1,1]])

if int_num == 2:
    int_sample0 = torch.Tensor([[-np.sqrt(1/3)],[np.sqrt(1/3)]])
    int_w0 = torch.Tensor([[1/2],[1/2]])
if int_num == 3:
    int_sample0 = torch.Tensor([[-np.sqrt(3/5)],[0],[np.sqrt(3/5)]])
    int_w0 = torch.Tensor([[5/18],[4/9],[5/18]])
if int_num == 4:
    int_sample0 = torch.Tensor([[-np.sqrt(1/3)],[-np.sqrt(1/3)],[np.sqrt(1/3)],[np.sqrt(1/3)]])
    int_w0 = torch.Tensor([[5/18],[4/9],[5/18]])

#设置积分的节点
int_sample = torch.zeros(int_sample_num*int_sample_num*int_num*int_num,2)
int_w = torch.zeros(int_sample_num*int_sample_num*int_num*int_num,1)
for i in range(int_sample_num):
    for l in range(int_sample_num):
        for j in range(int_num):
            for s in range(int_num):
                int_sample[i*int_sample_num*int_num*int_num+l*int_num*int_num+j*int_num+s,0] = -1 + h0*i+ int_sample0[j]*h0/2+h0/2
                int_sample[i*int_sample_num*int_num*int_num+l*int_num*int_num+j*int_num+s,1] = -1 + h0*l+ int_sample0[s]*h0/2+h0/2
                int_w[i*int_sample_num*int_num*int_num+l*int_num*int_num+j*int_num+s] = int_w0[j]*int_w0[s]*h0*h0
int_sample_num = int_sample_num*int_sample_num*int_num*int_num
                
sample = 625
zz = torch.linspace(-0.98,0.98,25)
x = torch.zeros(sample,2)
for i in range(25):
    for j in range(25):
        x[i*25+j,0],x[i*25+j,1] = zz[i],zz[j]
u_r = torch.rand(sample,1)
u_i = torch.rand(sample,1)
xy = torch.zeros(sample*int_sample_num,4)                 ### 坐标(x,y) 
H_r = torch.zeros(sample*int_sample_num,1)                  ### 即矩阵H(x,y)
H_i = torch.zeros(sample*int_sample_num,1)                  ### 即矩阵H(x,y)
for i in range(sample):
    xy[i*int_sample_num:(i+1)*int_sample_num,2:4]=int_sample
    xy[i*int_sample_num:(i+1)*int_sample_num,0:2]=x[i,:]

In [9]:
###  计算积分节点和边界积分点的相应的矩阵
G2_r_in = torch.zeros(int_sample_num,sample_num)
G2_i_in = torch.zeros(int_sample_num,sample_num)
for i in range(int_sample_num):
    for j in range(sample_num):
        j0 = int(j/M)
        j1 = int((j-1)/M)%vertex_num
        d = sample_x[j,:]-int_sample[i,:]
        r0 = d.norm()
        G2_r_in[i,j] = scp.hankel1(1,k*r0).imag/4*((d*normal[j0,:]*h[j0]+d*normal[j1,:]*h[j1]).sum())/2/r0*k 
        G2_i_in[i,j] = -scp.hankel1(1,k*r0).real/4*((d*normal[j0,:]*h[j0]+d*normal[j1,:]*h[j1]).sum())/2/r0*k 
        #G2_in[i,j] = -1/(2*np.pi)*(r*normal[j0,:]*h[j0]+r*normal[j1,:]*h[j1]).sum()/(2*d*d)

In [ ]:
###  计算H(x,y)
inxy = torch.zeros(sample_num,4)
inxy[:,2:4] = sample_x
for i in range(sample):
    inxy[:,0],inxy[:,1] = x[i,0],x[i,1]
    #sample_h = mynet1(inxy)
    sample_h_r,sample_h_i = (mynet1(inxy)[:,0]).reshape(-1,1) , (mynet1(inxy)[:,1]).reshape(-1,1)
    H_r[i*int_sample_num:(i+1)*int_sample_num] = -(G2_r_in@sample_h_r - G2_i_in@sample_h_i)
    H_i[i*int_sample_num:(i+1)*int_sample_num] = -(G2_i_in@sample_h_r + G2_r_in@sample_h_i)    

In [ ]:
###   f为被积的右端项 即 -laplace u - k^2u = f
###   此处积分计算的是 f H的积分，再加上f G_0的积分（G_0为基本解）即最后的结果

f_r = (2*(int_sample[:,0]**2+int_sample[:,1]**2-2)+k*k*(1-int_sample[:,0]**2)*(1-int_sample[:,1]**2)).reshape(-1,1)
f_i = 0
for i in range(sample):
    u_r[i] = (H_r[i*int_sample_num:(i+1)*int_sample_num]*f_r*int_w - H_i[i*int_sample_num:(i+1)*int_sample_num]*f_i*int_w).sum()
    u_i[i] = (H_r[i*int_sample_num:(i+1)*int_sample_num]*f_i*int_w + H_i[i*int_sample_num:(i+1)*int_sample_num]*f_r*int_w).sum()


1